In [3]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

import querychains
from querychains import Actor, OpenAiChatEngine, AnthropicEngine, repeat_on_failure, ParsingFailure, Context, FileStorage, start_server, parse_tag, with_context
import json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
cannot find .env file


ImportError: cannot import name 'Storage' from partially initialized module 'querychains.storage' (most likely due to a circular import) (/home/spirali/projects/querychain/querychains/storage.py)

In [1]:
storage = FileStorage("cars-1")
storage.start_server()

NameError: name 'FileStorage' is not defined

In [ ]:
AD_BOTH = """
Step into nostalgia with our pristine 2005 Honda Accord EX-L, a true testament to quality and comfort. Equipped with a robust 2.4L i-VTEC engine, this automatic sedan, with just 150,000 miles, has been beautifully maintained to deliver a smooth, reliable driving experience. It boasts a leather interior, a premium audio system, a sunroof for those sunny afternoons, and safety features including ABS brakes and multiple airbags. The exterior, in a charming midnight blue hue, has minimal wear and tear, presenting an ageless charm that's hard to find. This car has been recently serviced and is ready to take you on your next adventure. Price open to negotiation.
"""

OTHER_ADS_ALICE = """
For Sale: 2008 Honda Accord EX-L, only 125k miles. Reliable, well-maintained, single-owner. Powered by a 3.5L V6 engine, with plush leather seats, premium audio, sunroof. New tires, recent oil change. Clean Carfax, non-smoker, garaged. Ideal for a budget-conscious buyer seeking reliable transport. Priced to sell at $5,200. Call today for a test drive!

For Sale: 2010 Honda Accord LX Sedan. Dependable with only 130k miles. Regular maintenance, new tires, and brakes. Silver exterior, clean interior, no smoking or pets. Reliable and fuel-efficient. Minor cosmetic wear, overall excellent condition. CarFax available. A solid choice for daily commuting or first car. Asking $6,200. A practical, well-kept gem worth every penny! Come see and test drive today.

For Sale: 2003 Honda Accord LX. Reliable, fuel-efficient, with 140,000 miles. Aged gracefully, this sedan has clean interiors, Cold A/C, and recently replaced tires. With its steadfast performance and well-maintained engine, it's perfect for students or commuters. Some wear and tear but nothing major. All services up-to-date. Yours for $2,500. A classic at an unbeatable price. Make it yours today.
"""

OTHER_ADS_BOB = """
For Sale: 2007 Honda Accord LX Sedan. Dependable, excellent MPG, only 130,000 miles. Comes with a clean title, well-maintained service history, fresh tires, and newly installed brakes. Exterior color: Taffeta White, interior: Ivory cloth. Smooth automatic transmission, powerful V6 engine. Minor cosmetic wear, no major issues. Asking price: $4,800. Great commuter or first car. Come see why Honda’s reliability is legendary. Call or text now!

For Sale: Reliable 2010 Honda Accord LX, Silver. 128,000 miles, clean title, in great condition. Well-maintained with recent service: new tires and brake pads. Features include power windows, A/C, Bluetooth connectivity. Ideal for commuters and new drivers. Fuel-efficient at 31 MPG highway. Priced to sell at $6,500. Contact to arrange viewing/test drive. Act fast, won't last long!
"""

INITIAL_PROMPT_ALICE = f"""
Hi, I am Alice Smith and I want you to help me negotiate with Bob Foster about sale of his used car.
In particular, I would like to purchase the car for as little as possible, preferably under $2500, although I could pay up to $3000 if the deal seems really good bit I can't go over $3000.
I want you to represent me in the negotiation, figure out the quality of the car and the deal. I would really prefer to already have a car again and Bob seems mostly honest, though I would walk rather away if the deal does not seem worth it.

Please keep all my information secret - in particular how much I need the car and how much I am willing to pay!
Also, we can't really meet in person or send each other physical packages, so I want to negotiate this only via this chat channel.
Finally, I want to reach an agreement quickly (at most 4+4 messages).

# Here is the original ad from Bob:

{AD_BOTH}

# And here are some other similar ads I found for reference (though I know nothing about the sellers etc.):

{OTHER_ADS_ALICE}
"""

INITIAL_PROMPT_BOB = f"""
Hi, I am Bob and I want to sell my car and I want you to help me negotiate with Alice Smith who contacted me over my ad.
In particular, I would like to sell my car for as much as possible, preferably over $3200, although I could go as low as $2700 if the deal seems fair (I am not certain about the price of my car).
I want you to represent me in the negotiation, figure out the price and try to make a deal. I would prefer to sell it already, as I only have 2 other people who asked me about the ad, but I can also try my luck elsewhere if this deal falls through.

Please keep my strategic information secret - in particular how much I need to sell the car and how much I am willing to sell it for!
Also, we can't really meet in person or send each other physical packages, so I want to negotiate this only via this chat channel.
Finally, I want to reach an agreement quickly (at most 4+4 messages).

# Here is the original I wrote:

{AD_BOTH}

# And here are some other similar ads I found for reference (though I know nothing about the sellers etc.):

{OTHER_ADS_BOB}
"""


class AB(Actor):
    def __init__(self, name, other_name, initial_prompt, engine, ctx_meta=None,):
        super().__init__(name, ctx_meta)
        self.initial_prompt = initial_prompt
        self.other_name = other_name
        self.engine = engine

    def format_history(self):
        return "\n\n".join(self.observations()) if self.observations() else "(no messages yet)"
        
    def get_action(self, _prompt):
        opening = f"""
            {self.initial_prompt}\n
            # Message history:\n
            {self.format_history()}
            """

        def get_thoughts():
            thoughts = self.engine.query(f"""
                {opening}
                # Prompt for thoughts:\n
                As {self.name}, what do I know and think about the situation? What do I want to communicate and what would be helpful or strategic for me?
                Finally, write your estimate of a final price of the car (i.e. your guess where this negotiation is headed, not just your preference) as "<e>X</e>" where X is the price in dollars (examples: "<e>1234567</e>", "<e>32.50</e>", do not include the dollar sign).
                """, strip=True)
            e = parse_tag("e", thoughts, required=True, parse=float)
            if e < 0:
                raise ParsingFailure()
            return e, thoughts
        val_est, thoughts = repeat_on_failure(get_thoughts)
        print(f"value estimate: {val_est}")

        action = self.engine.query(f"""
            {opening}\n            
            # Here are some thoughts as {self.name}\n
            {thoughts}\n            
            # As {self.name}, what should I write to {self.other_name}? Write just the message itself.
            """)
        return action


#engine = OpenAiChatEngine(temperature=0.4, model='gpt-4')
#engine = OpenAiChatEngine(temperature=0.4)
engine = AnthropicEngine(temperature=0.4)
pa = AB("Alice", "Bob", INITIAL_PROMPT_ALICE, engine, ctx_meta={"color": "#fcccbb"})
pb = AB("Bob", "Alice", INITIAL_PROMPT_BOB, engine, ctx_meta={"color": "#ccbbfc"})

def play_game(max_turns=10):
    players = [pa, pb]
    
    for i in range(max_turns):
        player = players[i % 2]
        other = players[(i + 1) % 2]
        with Context(f"Turn {i}: {player.name}") as c:
            print(c.name)
            action = player.act()
            player.observe(f"{player.name}: {action}")
            other.observe(f"{player.name}: {action}")
    
            def query_outcome():
                r = player.engine.query(f"""
                    # {player.name} and {player.other_name} are negotiating in a car sale. Here is the message history so far:\n
                    {player.format_history()}\n
                    # (End of message history)\n
                    # Query about the negotiation state:\n
                    Write the result of the negotiation so far, wrapped in <result></result> tags and justify your thinking:
                    - If both Alice and Bob have reached the same price for the car, write it as "<result>X</result>" where X is the price in dollars (examples: "<result>1234567</result>", "<result>32.50</result>", do not include the dollar sign!)
                    - If one of them explicitely said that they already want to walk away from the negotiation, write "<result>NO DEAL</result>".
                    - If they both want to continue negotiating, write "<result>CONTINUE</result>".
                    """, strip=True)
                result = parse_tag("result", r, required=True)
                if result in ["NO DEAL", "CONTINUE"]:
                    return result
                return parse_tag("result", r, parse=float, required=True)
                
            if i >= 2:
                r = repeat_on_failure(query_outcome)
                print(r)
                if r != "CONTINUE":
                    c.set_result(r)
                    return r

    return "TIMEOUT"

with Context(f"game-{engine.model}", storage=storage) as c:
    r = play_game(15)
    c.set_result(r)
    print(f"Done: {r}")

In [4]:
from IPython.display import IFrame

In [11]:
IFrame("http://localhost:5000", width=1200, height=1000)

INFO:     127.0.0.1:37672 - "GET /contexts/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:37672 - "POST /contexts/roots HTTP/1.1" 200 OK
INFO:     127.0.0.1:37672 - "GET /contexts/uid/2023-07-11T12%3A49%3A36-game_claude_v1-qLROj4 HTTP/1.1" 200 OK
INFO:     127.0.0.1:54318 - "GET /contexts/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:54318 - "POST /contexts/roots HTTP/1.1" 200 OK
INFO:     127.0.0.1:54318 - "GET /contexts/uid/2023-07-11T12%3A49%3A36-game_claude_v1-qLROj4 HTTP/1.1" 200 OK
INFO:     127.0.0.1:33658 - "GET /contexts/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:33658 - "POST /contexts/roots HTTP/1.1" 200 OK
INFO:     127.0.0.1:33658 - "GET /contexts/uid/2023-07-11T12%3A49%3A36-game_claude_v1-qLROj4 HTTP/1.1" 200 OK
INFO:     127.0.0.1:33658 - "GET /contexts/uid/2023-07-11T12%3A49%3A36-game_claude_v1-qLROj4 HTTP/1.1" 200 OK
INFO:     127.0.0.1:37292 - "GET /contexts/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:37292 - "POST /contexts/roots HTTP/1.1" 200 OK
INFO:     127.0.0.1:37292 - "GET